In [6]:
using GLMakie
using JLD

In [7]:
function get_metadata(file_name::String)
    c = jldopen(file_name, "r") do file
        return read(file, "metadata")
    end
end

get_metadata (generic function with 1 method)

In [8]:
function load_matrix(timestep::Int, file_name::String, var::String)
    data_name = var * "_$timestep"
    c = jldopen(file_name, "r") do file
        return read(file, data_name)
    end
end

load_matrix (generic function with 1 method)

In [9]:
function animation()
    simulation_name = "test12"
    file_name = "../../data/$simulation_name.jld"
    
    metadata = get_metadata(file_name)
    println(metadata)

    T = floor(Int, metadata["T"])
    dt = metadata["dt"]
    sample_timestep = 4*metadata["sample_timestep"]
    sample_interval = metadata["sample_interval"]

    zeta = load_matrix(0, file_name, "zeta")

    zeta_top_plot = Observable(zeta[:,:,1])
    zeta_bottom_plot = Observable(zeta[:,:,2])

    # plot_time = Observable(0.0)

    fig = Figure()
    ga = fig[1, 1] = GridLayout()
    
    ax_top = Axis(ga[1, 1])
    ax_bottom = Axis(ga[2, 1])

    heatmap!(ax_top, zeta_top_plot)
    heatmap!(ax_bottom, zeta_bottom_plot)

    for (timestep, time) in enumerate(0:dt:T)
        if timestep % (sample_timestep) == 0
            zeta = load_matrix(Int(timestep), file_name, "zeta")
            
            # println("$file_name, $timestep, top | ", zeta[10,10,1])
            # println("$file_name, $timestep, bottom | ", zeta[10,10,2])

            zeta_top_plot[] = zeta[:,:,1]
            zeta_bottom_plot[] = zeta[:,:,2]     

            display(fig)

            # Set the matrix to nothing so the memoery can be freed up later by the garbage collector.
            matrix = nothing
            
            # Force garbage collection to clean up the memory used by the old array.
            GC.gc()
            sleep(0.2)
        end
    end
end

animation()

Dict{String, Real}("T" => 6.3072e7, "sample_timestep" => 72, "dt" => 1200.0, "sample_interval" => 86400.0)
